# ETL the data in the first out of a list of book reviews dataset from AWS S3 datasets in order to match the schema.

In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark
 
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"
 
# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# import postgres connection
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-09-22 00:55:58--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2020-09-22 00:55:59 (7.41 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonHWBook1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# load csv from AWS S3
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)

In [ ]:
# make header 'true' and sep '\t'
# verify data is connected
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-10-14 00:00:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

In [ ]:
# print df schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Count number of records(rows) in this dataset

In [ ]:
# count rows
row_number = df.count()
print(f"Number of records in this dataset: {row_number}")

Number of records in this dataset: 3105520


## Create user dataframe to match **review_id_table** table in the schema

In [ ]:
# make df for review
review_df = df.select(["review_id", "customer_id", "product_id", "product_parent","review_date"])
review_df.show(10)

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RQ58W7SMO911M|   12076615|0385730586|     122662979|2005-10-14 00:00:00|
|  RF6IUKMGL8SF|   12703090|0811828964|      56191234|2005-10-14 00:00:00|
|R1DOSHH6AI622S|   12257412|1844161560|     253182049|2005-10-14 00:00:00|
| RATOTLA3OF70O|   50732546|0373836635|     348672532|2005-10-14 00:00:00|
|R1TNWRKIVHVYOV|   51964897|0262181533|     598678717|2005-10-14 00:00:00|
|R2F53LI9KK9MOY|   31048862|0316769487|     862964341|2005-10-14 00:00:00|
|R1KJ6MB7MRSQFF|   53000124|0805076069|     145341889|2005-10-14 00:00:00|
|R2XIM9LT335WHE|   29732693|1581603681|     640542054|2005-10-14 00:00:00|
|R1VE0FQQ0QTQJN|   48541186|0300108834|     915891133|2005-10-14 00:00:00|
|R1VKEE2NWSWDRU|   15280864|0446531081|     880645124|2005-10-14 00:00:00|
+--------------+---------

In [ ]:
# double checking that datatype matches schema
review_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'timestamp')]

In [ ]:
# change review_date from timestamp to yyyy-mm-dd format and verify change
from pyspark.sql.types import DateType
review_df = review_df.withColumn("review_date", review_df['review_date'].cast(DateType()))
review_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

## Create user dataframe to match **products** table in the schema

In [ ]:
# make df for products and drop duplicates
product_df = df.select(["product_id", "product_title"]).drop_duplicates()
product_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0029148510|Acts of War: Beha...|
|1844494896|Waiting For Kate ...|
|0451216954|Dark Lover (Black...|
|006056718X|Often Wrong, Neve...|
|0965852768|101 Spooktacular ...|
|141960435X|Flip Flop Spanish...|
|0471282502|Experimental Orga...|
|0756613647|Universe: The Def...|
|B0006YGCE8|Seismic risk asse...|
|207030602X|L Etranger (Folio...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
# double checking that datatype matches schema
product_df.dtypes

[('product_id', 'string'), ('product_title', 'string')]

## Create user dataframe to match **customers** table in the schema

In [ ]:
# count the number of customers
customer = df.groupby('customer_id').agg({"customer_id": "count"})

#rename count(customer_id) column to match schema
customer_df = customer.withColumnRenamed('count(customer_id)', 'customer_count')
customer_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28735044|             9|
|   15607794|             1|
|   15596972|             1|
|   32371281|             2|
|   12084021|             1|
|   14462635|             1|
|   41052840|             5|
|   52068296|           105|
|   51739623|             1|
|   14544320|             1|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# double checking that datatype matches schema
customer_df.dtypes

[('customer_id', 'int'), ('customer_count', 'bigint')]

In [ ]:
# change customer_count from bigint to int & verify change
from pyspark.sql.types import IntegerType
customer_df = customer_df.withColumn("customer_count",customer_df["customer_count"].cast(IntegerType()))
customer_df.dtypes

[('customer_id', 'int'), ('customer_count', 'int')]

## Create user dataframe to match **vine_table** table in the schema

In [ ]:
# make df to match schema
vine_df = df.select(["review_id", "star_rating","helpful_votes","total_votes","vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RQ58W7SMO911M|          4|            2|          3|   N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|
|R1DOSHH6AI622S|          4|            1|         22|   N|
| RATOTLA3OF70O|          5|            2|          2|   N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|
|R2XIM9LT335WHE|          4|            3|         10|   N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
# double checking that datatype matches schema
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

## Load the DataFrames that correspond to tables into an RDS instance

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<insert endpoint>:5432/postgres"
config = {"user":"<insert username>", 
          "password": "<insert password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to review_df table in RDS
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to product_df table in RDS
product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customer_df table in RDS
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine_df table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

# Are Vine Reviews Trustworthy?

## A Comparison between Vine and Non-Vine Reviewers

Count vine members

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Find vine users
vine_users = vine_df.filter(col("vine")  == "Y")
vine_users.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2NKNL4WXWIWBG|          4|            3|          3|   Y|
| RAIG9ZQDMBF5P|          5|            3|          6|   Y|
+--------------+-----------+-------------+-----------+----+



In [ ]:
# count rows to verify
viners = vine_users.count()
print(f"Number of vine users in this dataset: {viners}")

Number of vine users in this dataset: 2


Count number of non-vine members

In [ ]:
# Find non-vine users
novine_users = vine_df.filter(col("vine")  == "N")
novine_users.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RQ58W7SMO911M|          4|            2|          3|   N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|
|R1DOSHH6AI622S|          4|            1|         22|   N|
| RATOTLA3OF70O|          5|            2|          2|   N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|
|R2XIM9LT335WHE|          4|            3|         10|   N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|
|R2UP6XSVYJBJ2H|          5|            0|          0|   N|
|R21SYDQ70ILUC0|          5|            1|          1|   N|
| R7M06Z88PD7SX|          4|           14|         14|   N|
| RRS38KZ4WB5O2|          5|            

In [ ]:
# count rows
no_viners = novine_users.count()
print(f"Number of non-vine users in this dataset: {no_viners}")

Number of non-vine users in this dataset: 3105513


In [ ]:
novine_users.groupBy("review_id").count()

DataFrame[review_id: string, count: bigint]

Total votes by star rating amongst non-vine users

In [ ]:
from pyspark.sql.functions import desc
novine_reviews_df = novine_users.select(["star_rating","total_votes"])\
         .groupBy("star_rating")\
         .agg({"total_votes": "count"})

novine_reviews_df.orderBy(desc("count(total_votes)")).show(truncate=False)

+-----------+------------------+
|star_rating|count(total_votes)|
+-----------+------------------+
|5          |1864803           |
|4          |586180            |
|3          |249925            |
|1          |238221            |
|2          |166384            |
+-----------+------------------+



In [ ]:
novine_tots_reviews = novine_users.select(["star_rating","total_votes", "helpful_votes"])\
         .groupBy("star_rating")\
         .agg({"total_votes": "sum", "helpful_votes": "sum"})

df1 = novine_tots_reviews.orderBy(desc("sum(total_votes)"))
df1.show(truncate=False)

+-----------+----------------+------------------+
|star_rating|sum(total_votes)|sum(helpful_votes)|
+-----------+----------------+------------------+
|5          |20824555        |17667965          |
|1          |6321283         |2965719           |
|4          |5524832         |4539419           |
|3          |3060700         |2139865           |
|2          |2777252         |1676047           |
+-----------+----------------+------------------+

